### PyTorch Lightning

PyTorch Lightning is an open source wrapper around PyTorch that abstracts away a lot of details, like training vs evaluation method, zeroing out gradients, using CUDA cores etc. It also comes with built in Tensorboard.

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch.nn.functional as F
from pytorch_lightning import Trainer

/Users/fayad/miniconda3/envs/ml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/fayad/miniconda3/envs/ml/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/Users/fayad/miniconda3/envs/ml/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/fayad/miniconda3/envs/ml/lib/python3.8/site-packages/torchvision/image.so
  Reason: tried: '/Users/fayad/miniconda3/envs/ml/lib/libpng16.16.dylib' (no such file), '/Users/fayad/miniconda3/envs/ml/lib/libpng16.16.dylib' (no such file), '/Users/fayad/miniconda3/envs/ml/lib/python3.8/lib-dynload/../../libpng16.16.dylib' (no such file), '/Users/fayad/miniconda3/envs/ml/lib/libpng16.16.dylib' (no such file), '/Users/fayad/minic

In [2]:
# Define hyperparameters
input_size = 784 # Images are 28x28 = 784 pixels each
hidden_size = 100 # Number of neurons in the hidden layer
num_classes = 10 # 10 classes or digits to be predicted
num_epochs = 2 # 2 epochs
batch_size = 100 # 100 samples per batch
alpha = 1e-3 # Learning rate = 0.001

In [3]:
# Create FFN
# We are going to be inheriting from pl.LightningModule instead of nn.Module
class FFN(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFN, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        return (self.l2(self.relu(self.l1(x))))
        # We are not using any softmax layer or one-hot encoding the output since we are going to be using CrossEntropyLoss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=alpha)
        return optimizer
    
    # PyTorch Lightning also requires a training_step, a configure_optimizers and a train_dataloader function
    # Additionally it is useful to include a validation_step and val_dataloader function for evaluation
    
    def training_step(self, batch, batch_idx):
        # Here we do not need to worry about the loops or the backprops
        
        # Acquire training sample from batch
        images, labels = batch
        # images are of the shape 100, 1, 28, 28
        # There are 100 images per batch
        # Each image has 1 channel (grayscale) and 28x28 pixels
        # The required format is 100, 784
        # 100 samples each with 784 features (pixel values)
        images = images.view(images.shape[0], 28*28) # Reshape the tensor, no need to push it to device
        
        # Forward propagation
        predicted_output = self(images) # self is model
        
        # Loss
        loss = F.cross_entropy(predicted_output, labels)
        tensorboard_logs = {'train_loss': loss}
        
        return {'train_loss': loss, 'log': tensorboard_logs}
    
    def train_dataloader(self):
        # Import dataset
        train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
        
        #Set up dataloader
        train_dataloader = torch.utils.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) # Could be useful to set num_workers attribute
        
        return train_dataloader
    
    def validation_step(self, batch, batch_idx):
        # Here we do not need to worry about the loops or the backprops
        
        # Acquire training sample from batch
        images, labels = batch
        # images are of the shape 100, 1, 28, 28
        # There are 100 images per batch
        # Each image has 1 channel (grayscale) and 28x28 pixels
        # The required format is 100, 784
        # 100 samples each with 784 features (pixel values)
        images = images.view(images.shape[0], 28*28) # Reshape the tensor, no need to push it to device
        
        # Forward propagation
        predicted_output = self(images) # self is model
        
        # Loss
        loss = F.cross_entropy(predicted_output, labels)
        return {'val_loss': loss}
    
    def val_dataloader(self):
        # Import dataset
        validation_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor(), download=True)
        
        #Set up dataloader
        validation_dataloader = torch.utils.DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False) # Could be useful to set num_workers attribute
        
        return validation_dataloader
    
    # It is also possible to define test_step and test_dataloader
    
    # It is also common practice to define a validation_epoch_end function
    # This gets called at the end of the validation epoch
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'avg_val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

In [4]:
# Finally start training
trainer = Trainer(fast_dev_run=True) 
# It is also posisble to add in more parameters, such as gpus, tpu_cores, callbacks, checkpoint_callbacks, max_epochs etc.
# fast_dev_run one epoch of training; helpful to check if everything is in order or not
model = FFN(input_size, hidden_size, num_classes)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 78.5 K
1 | relu | ReLU   | 0     
2 | l2   | Linear | 1.0 K 
--------------------------------
79.5 K    Trainable params
0         Non-trainable params
79.5 K    Total params
0.318     Total estimated model params size (MB)


Using downloaded and verified file: ./data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



 96%|███████████████████████████████████▍ | 27648/28881 [11:51<00:31, 38.88it/s]


/Users/fayad/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Refer to https://pytorch-lightning.readthedocs.io/en for more info